In [5]:
import random
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

In [39]:
dataset = pd.read_excel('data/test.xlsx')

In [25]:
dataset.head()

,URL,Title,Description,Article Text,Keywords,Tag
0,https://kino.mail.ru/news/61905-serial-na-viho...,Сериал на выходные: едкая детективная комедия ...,Рассказываем о густонаселенном сериале «Прести...,"В элитном поселке, где обитают сплошь богачи, ...","СТАТЬЯ, ЧТО ПОСМОТРЕТЬ, ДРАМА, КОМЕДИЯ, ДЕТЕКТИВ",Кино
1,https://kino.mail.ru/news/61913-film-na-vecher...,Фильм на вечер: атмосферный фильм-катастрофа с...,Погружаемся в апокалиптический триллер о франц...,Мир охвачен паникой — повсюду наблюдаются аном...,"СТАТЬЯ, ЧТО ПОСМОТРЕТЬ, ДРАМА, ТРИЛЛЕР, ФАНТАС...",Кино
2,https://kino.mail.ru/news/61900-treiler-dnya-m...,Трейлер дня: Милла Йовович возвращается в трил...,Актриса снялась в новом экшене про пост-апокал...,Вышел дублированный трейлер фантастического тр...,"СТАТЬЯ, ЧТО ПОСМОТРЕТЬ",Кино
3,https://kino.mail.ru/news/61871-film-na-vecher...,Фильм на вечер: милая зомби-комедия со звездой...,Заряжаемся хулиганским настроением за просмотр...,"Оставшись сиротой, юный Оливер оказывается в т...","СТАТЬЯ, ЧТО ПОСМОТРЕТЬ, КОМЕДИЯ, УЖАСЫ",Кино
4,https://kino.mail.ru/news/61854-treiler-dnya-7...,Трейлер дня: 79-летний Майкл Дуглас играет Бен...,В сериале «Франклин» актер в роли отца-основат...,Вышел трейлер костюмированной 8-серийной драмы...,"СТАТЬЯ, ЧТО ПОСМОТРЕТЬ",Кино


In [26]:
# Set a random seed
random_seed = 42
random.seed(random_seed)
 
# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [27]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [40]:
# Function to generate embeddings for each description
def generate_embeddings(description):
    # Tokenize and encode text
    encoding = tokenizer.encode_plus(
        description,               # Input text
        padding=True,             # Pad to the maximum sequence length
        truncation=True,          # Truncate to the maximum sequence length if necessary
        return_tensors='pt',      # Return PyTorch tensors
        add_special_tokens=True   # Add special tokens CLS and SEP
    )
    
    input_ids = encoding['input_ids']        # Token IDs
    attention_mask = encoding['attention_mask']  # Attention mask

    # Generate embeddings using BERT model
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        word_embeddings = outputs.last_hidden_state  # This contains the embeddings
    
    # Compute the average of word embeddings to get the sentence embedding
    sentence_embedding = word_embeddings.mean(dim=1)  # Average pooling along the sequence length dimension
    
    return sentence_embedding


In [41]:
%%time

# Apply the function to generate embeddings for each description
dataset['Description_embedding'] = dataset['Description'].apply(generate_embeddings)

CPU times: user 478 ms, sys: 411 ms, total: 889 ms
Wall time: 164 ms


In [43]:
dataset.Description_embedding[1].shape

torch.Size([1, 768])

In [44]:
res1 = generate_embeddings(['Я астронавт в космическом пространстве. Я постигаю мироздание и пью пиво.'])

In [38]:
res1.shape

torch.Size([1, 768])

In [ ]:
res2 = generate_embeddings(['Я астронавт в космическом пространстве. Я постигаю мироздание и пью пиво.'])